# Proposition d'éclairage
- **Note:** Impossible de remettre la main sur le site web qui décrit cette technique ! Si quelqu'un tombe dessus je suis preneur pour ajouter le lien en référence
- il s'agit de calculer pour chaque cellule l'angle de la lumière incidente qui parvient à la cellule.
- pour les besoins de l'exemple on utilise numpy (pour la représentation des tableaux et quelques calculs) et pandas (uniquement pour l'affichage dans ce notebook jupyter)

<br/>**ATTENTION: il s'agit simplement d'un POC sans aucune optimisation à part d'avoir séparer le calcul des angles de base (source lumineuse éclairant à 360° sans obstacle) de la prise en compte de la couverture lumineuse réelle et des obstacles [R1: mérite au moins une passe pour corriger le nommage et les commentaires (notamment clarifier, simplifier et corriger lumière incidente, angle, secteur, ...)]**

## Pourquoi ?
- parce qu'elle donne une idée de la visibilité d'une cellule prise dans sa globalité
- parce qu'elle permet d'avoir un rendu en niveau de gris
- parce que le niveau de visibilité d'une cellule (le ratio calculé plus bas) peut directement entrer en compte dans les calculs liés à la visée et au combat
- parce qu'elle est sans doute optimisable [R1: reste à vérifier à quel point]

## Modules et constantes
- juste le nécessaire
- LENGTH représente la longueur du tableau 2D représentant un quadrant éclairé
- les tableaux sont stockés lignes par lignes donc l'accès se fait via (y, x) 
- **note:** dans display, on change juste l'ordre d'affichage des ordonnées pour coller à une vision "humaine" classique

In [1]:
import numpy as np
import pandas as pd

# NOTE: array[y, x]
print("Imports: OK...")

Imports: OK...


In [2]:
# Constants
# size of arrays : LENGTHxLENGTH
WIDTH = 4
HEIGHT = 6

# if abs(a - b) < EPSILON then a == b
EPSILON = 0.1
# For approximation
ROUNDING_POSITION = 3

# Display Array, descending y axis
def display(msg, data):
    print(f"## {msg} ##\n {pd.DataFrame(data).sort_index(ascending=False, axis=0)}\n")

print("Constants and basic functions: OK...")

Constants and basic functions: OK...


## Calcul des valeurs de base
- le principe est très simple, on calcule l'angle de lumière parvenant à une cellule
- on a 3 vecteurs : 
  - vx: pour représenter l'axe des abscisses
  - a: le vecteur entre la source lumineuse et le point d'entrée dans la cellule
  - b: le vecteur entre la source lumineuse et le point de sortie de la cellule
![valeurs de base](images/schema01b.png "valeurs de base")
- on calcule: 
  - alpha l'angle de lumière perçue par la cellule
  - alpha_min et alpha_max les angles (vx,a) et (vx, b)
- les résultats obtenus sont stockés dans **base_alpha**, **base_alpha_min** et **base_alpha_max** pour une réutilisation ultérieure
- on utilise atan2 plutôt que acos pour des raisons de précision numérique   quans les angles deviennent petits [R1: j'ai lu ça quelque part mais  pas pris le temps de vérifier...]
- **computeBaseSector** sert à définir les vecteurs a et b en fonction de la position dans le tableau puis appelle **setBaseAlpha** pour le calcul
- **computeBaseSector** est une fonction récursive, il suffit de l'appeler sur la cellule du coin opposé à la source pour calculer les valeurs pour l'ensemble des cellules du quadrant.

In [3]:
# Light Source L (0.5, 0.5)
L = np.array([0.5, 0.5])
# x axis
vx = np.array([1., 0.])

# Compute base angles
base_alpha = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_min = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_max = np.zeros([HEIGHT, WIDTH])*np.nan

def setBaseAlpha(x, y, a, b):
    alpha = np.degrees(np.math.atan2(np.linalg.det([a,b]),np.dot(a,b)))
    beta = np.degrees(np.math.atan2(np.linalg.det([vx, a]), np.dot(vx, a)))
    #print(f"({x}, {y}) alpha:{alpha}, beta:{beta}")
    base_alpha[y, x] = alpha
    base_alpha_min[y, x] = beta
    base_alpha_max[y, x] = beta + alpha

def computeBaseSector():
    # First line
    y = 0
    for x in range(1,WIDTH):
        a = np.array([x, 0]) - L
        b = np.array([x, 1]) - L
        setBaseAlpha(x, y, a, b)
    
    # First Column
    x = 0
    for y in range(1, HEIGHT):
        a = np.array([1, y]) - L
        b = np.array([0, y]) - L
        setBaseAlpha(x, y, a, b)

    # All the other cells
    for y in range(1, HEIGHT):
        for x in range(1, WIDTH):
            a = np.array([x+1, y]) - L
            b = np.array([x, y+1]) - L
            setBaseAlpha(x, y, a, b)

computeBaseSector()
display("Base Alpha", base_alpha)
display("Base Alpha Min", base_alpha_min)
display("Base Alpha Max", base_alpha_max)


## Base Alpha ##
            0          1          2          3
5  12.680383  13.240520  13.799485  13.431029
4  16.260205  16.858399  17.102729  15.945396
3  22.619865  22.833654  21.801409  18.924644
2  36.869898  33.690068  28.072487  21.801409
1  90.000000  53.130102  33.690068  22.833654
0        NaN  90.000000  36.869898  22.619865

## Base Alpha Min ##
            0          1          2          3
5  83.659808  71.565051  60.945396  52.125016
4  81.869898  66.801409  54.462322  45.000000
3  78.690068  59.036243  45.000000  35.537678
2  71.565051  45.000000  30.963757  23.198591
1  45.000000  18.434949  11.309932   8.130102
0        NaN -45.000000 -18.434949 -11.309932

## Base Alpha Max ##
             0          1          2          3
5   96.340192  84.805571  74.744881  65.556045
4   98.130102  83.659808  71.565051  60.945396
3  101.309932  81.869898  66.801409  54.462322
2  108.434949  78.690068  59.036243  45.000000
1  135.000000  71.565051  45.000000  30.963757
0         

## Calcul avec une source lumineuse spécifique
- on cherche à connaitre l'angle de la source lumineuse effectivement perçu par une cellule en fonction de la couverture de la source lumineuse
- A part la cellule contenant la source lumineuse, toutes les cellules Cx,y reçoivent leur lumière en fonction de ce qui est transmis par 2 cellules voisines Cx-1,y et Cx,y-1
- si on note Ax,y l'angle de lumière reçue par la cellule Cx,y et Bx,y l'angle maximal théorique que peut recevoir la cellule (celui calculé à l'étape précédente) on a : Ax,y = (Ax-1,y union Ax,y-1) inter Bx,y
- dans le schéma suivant, les cellules sont notés A, B et C. l'indice 'r' correspond aux données réelles calculées suivant les caractéristiques de la source lumineuse ; 'b' correspond aux données de base. [R1: ouais, désolé deux lignes, deux notations différentes!]
![intersection](images/schema02.png "Intersection: b base, r réel")
- **real_alpha**, **real_alpha_min** et **real_alpha_max** contiennent les résultats de cette étape
- **intersect, union** : comme un obstacle occupe une cellule complète, il ne peut pas y avoir de trou dans l'interval après (Ax-1,y union Ax,y-1) inter Bx,y. Comme on ne veut jamais gérer un éventuel trou, ce qui est fait est : Ax,y = (Ax-1,y inter Bx,y) union (Ax,y-1 inter Bx,y).

In [4]:
# Source light angles (min, max)
light_angles = np.array([-45., 45.])

real_alpha_min = np.zeros([HEIGHT, WIDTH])*np.nan
real_alpha_max = np.zeros([HEIGHT, WIDTH])*np.nan
real_alpha = np.zeros([HEIGHT, WIDTH])

real_alpha_min[0, 0] = light_angles[0]
real_alpha_max[0, 0] = light_angles[1]
real_alpha[0, 0] = light_angles[1] - light_angles[0]

# Get the intersection of 2 angles
def intersect(in_min, in_max, th_min, th_max):
    if (in_max < th_min or in_min > th_max):
        # No overlap
        return (0., 0.)
    r_max = min(th_max, in_max)
    r_min = max(th_min, in_min)
    if (r_max - r_min < EPSILON):
        r_max = r_min
    return (r_min, r_max)

# Get the union of 2 angles IF they touch each other
def union(a1_min, a1_max, a2_min, a2_max):
    # One angle is null
    if (a1_min == a1_max == 0):
        return(a2_min, a2_max)
    if (a2_min == a2_max == 0):
        return(a1_min, a1_max)
    # Not neighbors ?
    if (abs(a2_min - a1_max) > EPSILON and abs(a1_min - a2_max) > EPSILON):
        #print(a1_min, a1_max, a2_min, a2_max, abs(a2_min - a1_max))
        print(f"[union] - FATAL - ({x}, {y}) ({a1_min}, {a1_max}) not neighbor of ({a2_min}, {a2_max})")
        return (0., 0.)
    # Union
    return (min(a1_min, a2_min), max(a1_max, a2_max))


def setRealAlpha(x, y, a_min, a_max):
    real_alpha_min[y, x] = a_min
    real_alpha_max[y, x] = a_max
    real_alpha[y, x] = a_max - a_min
    return(real_alpha_min[y, x], real_alpha_max[y, x])

# For each cell, compute the incident light
def computeSector(x, y):
    # Already done
    if (not np.isnan(real_alpha_min[y, x]) and not np.isnan(real_alpha_max[y, x])):
        return (real_alpha_min[y, x], real_alpha_max[y, x])
    # Origin
    if (x == 0 and y == 0):
        return (real_alpha_min[0, 0], real_alpha_max[0, 0])
    # First row (only x-1 is needed)
    if (y == 0):
        prevx = computeSector(x-1, y)
        real = intersect(prevx[0], prevx[1], base_alpha_min[y, x], base_alpha_max[y, x])
        return setRealAlpha(x, y, real[0], real[1])
    # First column (only y-1 is needed)
    if (x == 0):
        prevy = computeSector(x, y-1)
        real = intersect(prevy[0], prevy[1], base_alpha_min[y, x], base_alpha_max[y, x])
        return setRealAlpha(x, y, real[0], real[1])
    else:
        # The other cells (x-1 AND y-1 control the flow of light entering this cell)
        prevx = computeSector(x-1, y)
        prevy = computeSector(x, y-1)
        real = intersect(prevx[0], prevx[1], base_alpha_min[y, x], base_alpha_max[y, x])
        real2 = intersect(prevy[0], prevy[1], base_alpha_min[y, x], base_alpha_max[y, x])
        real = union(real[0], real[1], real2[0], real2[1])
        return setRealAlpha(x, y, real[0], real[1])

computeBaseSector()
computeSector(WIDTH-1, HEIGHT-1)

print(f"Light source angles ({light_angles[0]}, {light_angles[1]})\n")

display("Real Alpha Min", real_alpha_min)
display("Real Alpha Max", real_alpha_max)
display("Real Alpha", real_alpha)

Light source angles (-45.0, 45.0)

## Real Alpha Min ##
       0          1          2          3
5   0.0   0.000000   0.000000   0.000000
4   0.0   0.000000   0.000000  45.000000
3   0.0   0.000000  45.000000  35.537678
2   0.0  45.000000  30.963757  23.198591
1  45.0  18.434949  11.309932   8.130102
0 -45.0 -45.000000 -18.434949 -11.309932

## Real Alpha Max ##
       0     1          2          3
5   0.0   0.0   0.000000   0.000000
4   0.0   0.0   0.000000  45.000000
3   0.0   0.0  45.000000  45.000000
2   0.0  45.0  45.000000  45.000000
1  45.0  45.0  45.000000  30.963757
0  45.0  45.0  18.434949  11.309932

## Real Alpha ##
       0          1          2          3
5   0.0   0.000000   0.000000   0.000000
4   0.0   0.000000   0.000000   0.000000
3   0.0   0.000000   0.000000   9.462322
2   0.0   0.000000  14.036243  21.801409
1   0.0  26.565051  33.690068  22.833654
0  90.0  90.000000  36.869898  22.619865



## Obstacles
- **blocks** permet de positionner les obstacles (attention au format en ligne (y, x)
- **real_ratio** permet de savoir quel est le rapport entre la lumière réellement reçue par la cellule (en fonction des obstacles et de la couverture de la source lumineuse) et la lumière qu'elle aurait pu percevoir sans aucune contrainte
- ce ratio sera utilisé à l'affichage
- Deux points particuliers dans les cacluls (**setRealAlpha**):
  - des approximations dans les calculs peuvent conduire à un ratio de 0.9999... alors qu'en réalité il est de 1.0
  - pour les cellules bloquantes, on garde uniquement le ratio pour l'affichage et on réinitialise les autres valeurs (real_alpha* à 0 pour bloquer la propagation de la lumière)

In [5]:
light_angles = np.array([-180., 180.])

# Blocking blocks
blocks = np.zeros([HEIGHT, WIDTH])
blocks[2, 1] = 1

# [0. ; 1.] ratio = entering light angle / base light angle
# are we receiving all the lighting that we can accept without blocking element?
real_ratio = np.zeros([HEIGHT, WIDTH])

def resetArrays():
    real_alpha_min.fill(np.nan)
    real_alpha_max.fill(np.nan)
    real_alpha.fill(0.)
    real_ratio.fill(0.)
    real_alpha_min[0, 0] = light_angles[0]
    real_alpha_max[0, 0] = light_angles[1]
    real_alpha[0, 0] = light_angles[1] - light_angles[0]
    real_ratio[0, 0] = 1.

def setRealAlpha(x, y, a_min, a_max):
    real_alpha_min[y, x] = a_min
    real_alpha_max[y, x] = a_max
    real_alpha[y, x] = a_max - a_min
    # Watch out for the 1.0 -> 0.9999... 1.0 effect
    real_ratio[y, x] = round(real_alpha[y, x] / base_alpha[y, x], ROUNDING_POSITION)
#    if (1. - real_ratio[y,x]) < EPSILON:
#        real_ratio[y, x] = 1.
    # if we block lighting keep only the ratio and reset all other elements
    # with the ratio we know if the blocking cell receive light (obstacle must be displayed)
    if (blocks[y, x] == 1):
        real_alpha[y, x] = 0.
        real_alpha_min[y, x] = 0.
        real_alpha_max[y, x] = 0.
    return(real_alpha_min[y, x], real_alpha_max[y, x])

# For each cell, compute the incident light
def computeSector(x, y):
    # Already done
    if (not np.isnan(real_alpha_min[y, x]) and not np.isnan(real_alpha_max[y, x])):
        return (real_alpha_min[y, x], real_alpha_max[y, x])
    # Origin
    if (x == 0 and y == 0):
        return (real_alpha_min[0, 0], real_alpha_max[0, 0])
    # First row (only x-1 is needed)
    if (y == 0):
        prevx = computeSector(x-1, y)
        real = intersect(prevx[0], prevx[1], base_alpha_min[y, x], base_alpha_max[y, x])
        return setRealAlpha(x, y, real[0], real[1])
    # First column (only y-1 is needed)
    if (x == 0):
        prevy = computeSector(x, y-1)
        real = intersect(prevy[0], prevy[1], base_alpha_min[y, x], base_alpha_max[y, x])
        return setRealAlpha(x, y, real[0], real[1])
    # The other cells (x-1 AND y-1 control the flow of light entering this cell)
    else:
        prevx = computeSector(x-1, y)
        prevy = computeSector(x, y-1)
        real = intersect(prevx[0], prevx[1], base_alpha_min[y, x], base_alpha_max[y, x])
        real2 = intersect(prevy[0], prevy[1], base_alpha_min[y, x], base_alpha_max[y, x])
#        if (x == 3 and y == 2):
#            print(f'({x}, {y}) prevx({prevx[0]}; {prevx[1]}) prevy({prevy[0]}; {prevy[1]})')
#            print(f'({x}, {y}) after prevx intersect real({real[0]}; {real[1]})')
#            print(f'({x}, {y}) after prevy intersect real2({real2[0]}; {real2[1]})')
        real = union(real[0], real[1], real2[0], real2[1])
#        if (x == 3 and y == 2):
#            print(f'({x}, {y}) after final union real({real[0]}; {real[1]})')
        return setRealAlpha(x, y, real[0], real[1])

resetArrays()
#computeBaseSector()
computeSector(WIDTH-1, HEIGHT-1)

print(f"Light source angles ({light_angles[0]}, {light_angles[1]})\n")

display("Real Alpha Min", real_alpha_min)
display("Real Alpha Max", real_alpha_max)
display("Real Alpha", real_alpha)
display("Base Alpha", base_alpha)
display("Real Ratio", real_ratio)

Light source angles (-180.0, 180.0)

## Real Alpha Min ##
             0          1          2          3
5   83.659808  78.690068   0.000000   0.000000
4   81.869898  78.690068   0.000000  45.000000
3   78.690068  78.690068  45.000000  35.537678
2   71.565051   0.000000  30.963757  23.198591
1   45.000000  18.434949  11.309932   8.130102
0 -180.000000 -45.000000 -18.434949 -11.309932

## Real Alpha Max ##
             0          1          2          3
5   96.340192  84.805571   0.000000   0.000000
4   98.130102  83.659808   0.000000  45.000000
3  101.309932  81.869898  45.000000  45.000000
2  108.434949   0.000000  45.000000  45.000000
1  135.000000  71.565051  45.000000  30.963757
0  180.000000  45.000000  18.434949  11.309932

## Real Alpha ##
             0          1          2          3
5   12.680383   6.115504   0.000000   0.000000
4   16.260205   4.969741   0.000000   0.000000
3   22.619865   3.179830   0.000000   9.462322
2   36.869898   0.000000  14.036243  21.801409
1   90

## Expérimentation sur la visualisation
- un POC quick'n dirty pour se donner une idée du rendu en dégradé de gris sur les calculs précédents
- utilise COLOR_RANGE+1 niveaux de gris de (COLOR_MIN à COLOR_MAX)
- le noir est réservé pour les cellules qui ne reçoivent aucune lumière
- le niveau de gris le plus clair est réservé aux cellules dont le ratio est 1.0 (aucune déperdition)... **_[R1: A VOIR...]_**
- la valeur de la couleur obtenu dans **convertLightingRatioToColor** est utilisé pour alimenter les trois canaux RGB
- **note:** théoriquement, le dernier tableau affiché dans cette page devrait refléter cette valeur dans la couleur de fond de chaque cellule

In [6]:
# Constants
# COLOR_RANGE+1 shades of grey
COLOR_RANGE = 4
COLOR_MIN = 40
COLOR_MAX = 200
COLOR_DELTA = int((COLOR_MAX - COLOR_MIN) / COLOR_RANGE)

lighting_color = np.zeros([HEIGHT, WIDTH])

def convertLightingRatioToColor():
    for y in range(HEIGHT):
        for x in range(WIDTH):
            if (real_ratio[y, x] == 0.):
                lighting_color[y, x] = 0
            else:
                lighting_color[y, x] = int(abs(real_ratio[y, x]) * COLOR_RANGE) * COLOR_DELTA + COLOR_MIN

def color_max_red(value):
    v = int(value)
    hexcolor = '#%02x%02x%02x' % (v, v, v)
    return 'background-color: {}'.format(hexcolor)

def color_block(value):
    return 'font-weight: bold; color: red'

def markBlocks(datastyle):
    for y in range(HEIGHT):
        for x in range(WIDTH):
            if (blocks[y, x]):
                datastyle.applymap(color_block, subset=(y, x))
                
convertLightingRatioToColor()

print(f"Light source angles ({light_angles[0]}, {light_angles[1]})\n")

display("cell lighting value", lighting_color)

df = pd.DataFrame(lighting_color).sort_index(ascending=False, axis=0)
s = df.style.applymap(color_max_red)
markBlocks(s)
display("blocks", blocks)
s



Light source angles (-180.0, 180.0)

## cell lighting value ##
        0      1      2      3
5  200.0   80.0    0.0    0.0
4  200.0   80.0    0.0    0.0
3  200.0   40.0    0.0  120.0
2  200.0  200.0  120.0  200.0
1  200.0  200.0  200.0  200.0
0  200.0  200.0  200.0  200.0

## blocks ##
      0    1    2    3
5  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0
2  0.0  1.0  0.0  0.0
1  0.0  0.0  0.0  0.0
0  0.0  0.0  0.0  0.0



,0,1,2,3
5,200,80,0,0
4,200,80,0,0
3,200,40,0,120
2,200,200,120,200
1,200,200,200,200
0,200,200,200,200


### Cas murs sur diagonale
- pas de lumière filtrant entre 2 murs accolés

In [7]:
# Blocking blocks
blocks = np.zeros([HEIGHT, WIDTH])
blocks[1, 2] = 1
blocks[2, 1] = 1

resetArrays()
computeSector(WIDTH-1, HEIGHT-1)
convertLightingRatioToColor()

print(f"Light source angles ({light_angles[0]}, {light_angles[1]})\n")

display("cell lighting value", lighting_color)

df = pd.DataFrame(lighting_color).sort_index(ascending=False, axis=0)
s = df.style.applymap(color_max_red)
markBlocks(s)
display("blocks", blocks)
s


Light source angles (-180.0, 180.0)

## cell lighting value ##
        0      1      2      3
5  200.0   80.0    0.0    0.0
4  200.0   80.0    0.0    0.0
3  200.0   40.0    0.0    0.0
2  200.0  200.0    0.0    0.0
1  200.0  200.0  200.0   40.0
0  200.0  200.0  200.0  200.0

## blocks ##
      0    1    2    3
5  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0
2  0.0  1.0  0.0  0.0
1  0.0  0.0  1.0  0.0
0  0.0  0.0  0.0  0.0



,0,1,2,3
5,200,80,0,0
4,200,80,0,0
3,200,40,0,0
2,200,200,0,0
1,200,200,200,40
0,200,200,200,200
